In [2]:
import os
import subprocess

# Define the base directory containing the runs
base_dir = '/home/sl221120/WildfireSpreadTS/lightning_logs/wildfire_progression'
data_dir = '/home/sl221120/sl636/WildfireSpreadTS_HDF5'

# Log data mapping run_id to data_fold_id
log_data = [
    {"run_id":"4adsj08p", "data_fold_id":0},
    {"run_id":"heg3b8sr", "data_fold_id":1},
    {"run_id":"g4kcwn56", "data_fold_id":2},
    {"run_id":"7t9l4h3a", "data_fold_id":3},
    {"run_id":"cgce7y3v", "data_fold_id":4},
    {"run_id":"h495vls8", "data_fold_id":5},
    {"run_id":"e90loozf", "data_fold_id":6},
    {"run_id":"81azfn75", "data_fold_id":7},
    {"run_id":"p8y7pgep", "data_fold_id":8},
    {"run_id":"a1cyprhl", "data_fold_id":9},
    {"run_id":"ju2yiuod", "data_fold_id":10},
    {"run_id":"w09hg1ub", "data_fold_id":11},
]

# Convert log data to a dictionary
run_id_to_fold_id = {entry["run_id"]: entry["data_fold_id"] for entry in log_data}

# Function to run the test command for a specific checkpoint
def run_test(checkpoint_path, data_fold_id):
    test_command = [
        'python',
        '/data/users/sl636/WildfireSpreadTS/src/train.py',
        '--config=cfgs/unet/res18_monotemporal_test_only.yaml',
        '--trainer=cfgs/trainer_single_gpu.yaml',
        '--data=cfgs/data_monotemporal_full_features.yaml',
        '--seed_everything=0',
        '--trainer.max_epochs=200',
        '--do_test=True',
        f'--data.data_fold_id={data_fold_id}',
        f'--data.data_dir={data_dir}'
        f'--config.ckpt_path={checkpoint_path}'
    ]

    # Run the test command
    subprocess.run(test_command)

# Iterate through each run folder
for run_id, data_fold_id in run_id_to_fold_id.items():
    run_path = os.path.join(base_dir, run_id, 'checkpoints')
    # Find the checkpoint file with the best validation loss
    checkpoint_file = os.listdir(run_path)[0]
    if checkpoint_file.endswith(".ckpt"):
        run_test(checkpoint_file, data_fold_id)


python: can't open file '/data/users/sl636/WildfireSpreadTS/src/train.py': [Errno 2] No such file or directory
python: can't open file '/data/users/sl636/WildfireSpreadTS/src/train.py': [Errno 2] No such file or directory
python: can't open file '/data/users/sl636/WildfireSpreadTS/src/train.py': [Errno 2] No such file or directory
python: can't open file '/data/users/sl636/WildfireSpreadTS/src/train.py': [Errno 2] No such file or directory
python: can't open file '/data/users/sl636/WildfireSpreadTS/src/train.py': [Errno 2] No such file or directory
python: can't open file '/data/users/sl636/WildfireSpreadTS/src/train.py': [Errno 2] No such file or directory
python: can't open file '/data/users/sl636/WildfireSpreadTS/src/train.py': [Errno 2] No such file or directory
python: can't open file '/data/users/sl636/WildfireSpreadTS/src/train.py': [Errno 2] No such file or directory
python: can't open file '/data/users/sl636/WildfireSpreadTS/src/train.py': [Errno 2] No such file or directory
p

In [3]:
log_data = [
    {"run_id":"4adsj08p", "data_fold_id":0},
    {"run_id":"heg3b8sr", "data_fold_id":1},
    {"run_id":"g4kcwn56", "data_fold_id":2},
    {"run_id":"7t9l4h3a", "data_fold_id":3},
    {"run_id":"cgce7y3v", "data_fold_id":4},
    {"run_id":"h495vls8", "data_fold_id":5},
    {"run_id":"e90loozf", "data_fold_id":6},
    {"run_id":"81azfn75", "data_fold_id":7},
    {"run_id":"p8y7pgep", "data_fold_id":8},
    {"run_id":"a1cyprhl", "data_fold_id":9},
    {"run_id":"ju2yiuod", "data_fold_id":10},
    {"run_id":"w09hg1ub", "data_fold_id":11},
]

# Convert log data to a dictionary
run_id_to_fold_id = {entry["run_id"]: entry["data_fold_id"] for entry in log_data}


run_id_to_fold_id.items()

dict_items([('4adsj08p', 0), ('heg3b8sr', 1), ('g4kcwn56', 2), ('7t9l4h3a', 3), ('cgce7y3v', 4), ('h495vls8', 5), ('e90loozf', 6), ('81azfn75', 7), ('p8y7pgep', 8), ('a1cyprhl', 9), ('ju2yiuod', 10), ('w09hg1ub', 11)])

In [4]:
import pandas as pd 

def compute_ap(file_path):
    df = pd.read_csv(file_path)

    # Compute the average and standard deviation of test_AP
    test_ap_mean = df['test_AP'].mean()
    test_ap_std = df['test_AP'].std()


    print(test_ap_mean, test_ap_std)


In [2]:
file_path = '/home/sl221120/WildfireSpreadTS/src/veg_mono.csv'
print("veg_mono")
compute_ap(file_path)

veg_mono
0.3228241143127282 0.09496183157932543


In [3]:
file_path = '/home/sl221120/WildfireSpreadTS/src/multi_mono.csv'
print("multi_mono")
compute_ap(file_path)



multi_mono
0.33916346604625386 0.09063526435030687


In [4]:
file_path = '/home/sl221120/WildfireSpreadTS/src/all_mono.csv'
print("all_mono")
compute_ap(file_path)


all_mono
0.32728181034326553 0.10340745934365056


In [6]:
file_path = '/home/sl221120/WildfireSpreadTS/src/veg_multi.csv'
print("veg_multi")
compute_ap(file_path)

veg_multi
0.3335219609240691 0.08486465050508904


In [7]:
file_path = '/home/sl221120/WildfireSpreadTS/src/multi_multi.csv'
print("multi_multi")
compute_ap(file_path)

multi_multi
0.3407648255427678 0.08584146476653304


In [6]:
file_path = '/home/sl221120/WildfireSpreadTS/src/all_multi.csv'
print("all_multi")
compute_ap(file_path)

all_multi
0.29986127962668735 0.10934251644823732


In [7]:
file_path = '/home/sl221120/WildfireSpreadTS/src/all_multi_new.csv'
print("all_multi")
compute_ap(file_path)

all_multi
0.29986127962668735 0.10934251644823731
